In [106]:
import pandas as pd 
import numpy as np 
import requests

In [107]:
# URL de la API con los parámetros de fecha (1 y 2 de marzo)
url = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"


In [108]:
# Realizar la solicitud GET
response = requests.get(url)
response.status_code

200

In [109]:
data = response.json() # convierte la respuesta en un json 
data

{'@context': {'c': 'http://www.w3.org/2002/12/cal#',
  'dcterms': 'http://purl.org/dc/terms/',
  'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
  'loc': 'http://purl.org/ctic/infraestructuras/localizacion#',
  'org': 'http://purl.org/ctic/infraestructuras/organizacion#',
  'vcard': 'http://www.w3.org/2006/vcard/ns#',
  'schema': 'https://schema.org/',
  'title': 'vcard:fn',
  'id': 'dcterms:identifier',
  'relation': 'dcterms:relation',
  'references': 'dcterms:references',
  'address': 'vcard:adr',
  'area': 'loc:barrio',
  'district': 'loc:distrito',
  'locality': 'vcard:locality',
  'postal-code': 'vcard:postal-code',
  'street-address': 'vcard:street-address',
  'location': 'vcard:geo',
  'latitude': 'geo:lat',
  'longitude': 'geo:long',
  'organization': 'vcard:org',
  'organization-desc': 'dcterms:description',
  'accesibility': 'org:accesibilidad',
  'services': 'org:servicios',
  'schedule': 'org:horario',
  'organization-name': 'vcard:organization-name',
  'description': '

In [110]:
# Imprime las claves principales del JSON para inspeccionarlas
print("Claves principales del JSON:", data.keys())

Claves principales del JSON: dict_keys(['@context', '@graph'])


In [111]:
data['@graph'][0].keys()

dict_keys(['@id', '@type', 'id', 'title', 'description', 'free', 'price', 'dtstart', 'dtend', 'time', 'excluded-days', 'recurrence', 'uid', 'link', 'event-location', 'references', 'relation', 'address', 'location', 'organization'])

In [112]:
titulo = data.get('@graph')[0].get('title')
titulo

'25º aniversario de la revista La Fragua'

In [113]:
cod_postal = data.get('@graph')[0].get('address').get('area').get('postal-code')
cod_postal

'28005'

 El json obtenido es un diccionario que cuenta con dos claves principales, que son '@context', '@graph'. Nosotros necesitamos acceder al valor de la clave '@graph', que es una lista formada por un único elemento, que es un diccionario. Dentro de este diccionario encontramos ya diversas claves de donde vamos a obtener valores, como por ejemplo: necesito obtener el el nombre_evento que se va a obtener de 'title', la url_evento que se va a obtener de 'link' y horario que se va a obtener de 'dtstart' y 'dtend'. Por otro lado, en algunos casos, habrá que acceder al valor de otras claves para poder obtener la información que queremos. Por ejemplo, para la organizacion habrá que acceder en primer lugar al valor de la clave 'organization', que es un diccionario, y sobre ese diccionario acceder al valor de la clave 'organization_name'. También ocurre con el codigo postal o la direccion. En estos caso hay que acceder en primer lugar al valor de la clave 'address', que es otro diccionario y dentro de este diccionario acceder al valor de la clave 'area' que es otro diccionario, y por último, dentro de este último diccionario acceder al valor de la clave 'postal-code' / 'street-address'. El id del evento será generado en el momento de carga de la tabla a la base de datos, pues al ser un id seriado se realiza de forma automática. 

In [114]:
info_eventos = {
    "nombre_evento": [],
    "url_evento": [],
    "codigo_postal": [],
    "direccion": [],
    "horario": [],
    "organizacion": []
}

for evento in data["@graph"]:
        info_eventos["nombre_evento"].append(evento.get('title'))
        info_eventos["url_evento"].append(evento.get('link'))
        address = evento.get("address", {})
        area = address.get("area", {}) 
        info_eventos["codigo_postal"].append(area.get("postal-code", np.nan))
        info_eventos["direccion"].append(area.get("street-address", np.nan))
        info_eventos["horario"].append("Del " + evento.get('dtstart').split(".")[0][:16] + " al " + evento.get('dtend').split(".")[0][:16])
        organizacion = evento.get('organization', {})
        info_eventos["organizacion"].append(organizacion.get('organization-name', np.nan))

info_eventos


{'nombre_evento': ['25º aniversario de la revista La Fragua',
  '3CLONWS',
  'A la fresca',
  'A Magic Treasure',
  'A toda vela',
  'A verso y voz',
  'Academia de hadas y duendes',
  'Academia de hadas y duendes',
  'Acompañamiento digital a personas mayores',
  'Acompañar en la pérdida',
  'Activas en compañía: gimnasia de mantenimiento',
  'Actividades del Centro Cultural Casa del Reloj',
  "Actividades en el centro juvenil 'Carabanchel Alto'. Febrero 2024",
  "Actividades en el centro juvenil 'El Aleph' (Villa de Vallecas). Febrero 2024",
  "Actividades en el centro juvenil 'El Sitio de mi recreo' (Villa de Vallecas). Febrero  2025",
  "Actividades en el centro juvenil 'Hontalbilla' (Fuencarral - El Pardo). Febrero 2025",
  "Actividades en el centro juvenil 'Ouka Leele'. Febrero 2025.",
  "Actividades en el centro juvenil 'Pipo Velasco'. Febrero 2025",
  'Actividades mensuales de los centros de mayores del distrito Hortaleza',
  "Actuación de guiñol con animación y malabares: 'ANA

In [115]:
fecha = data.get('@graph')[0].get('dtstart')
fecha_convertida = pd.to_datetime(fecha)
fecha_convertida

Timestamp('2025-02-24 00:00:00')

In [116]:
df_raw = pd.read_parquet("../data/reservas_hoteles.parquet")
df_raw.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
584,ed8bf47b-63b7-45a3-ba3a-c1cf7259423b,5d06080f-f4a1-46ec-95ac-6ecf4dff6f0d,Ofelia,Sáenz,ofelia.sáenz@example.com,False,2025-02-04,2025-03-01,2025-03-02,29,497.35,Hotel Encanto Real,4.0,Madrid
7698,495e4276-c94b-41e1-86b7-854bf58d2f2c,0898cabf-9d33-402e-bace-7fee9fbaeebb,Conrado,Pizarro,conrado.pizarro@example.com,False,2025-02-03,None,None,29,NaN,Palacio del Sol,2.0,Madrid
2027,065f09b8-0537-4254-bb0a-f39d5899ba22,3fad6780-c9f9-4e18-b873-7c14bffc77fc,Tecla,Naranjo,tecla.naranjo@example.com,False,2025-02-06,2025-03-01,2025-03-02,38,426.63,Hotel Vista Alegre,1.0,Madrid
2641,236b0a1b-f3ba-4b7a-bcac-8db247d0920f,942c5731-9cb5-470f-bea8-f2ae1a66f119,Ángeles,Ropero,ángeles.ropero@example.com,False,2025-02-04,2025-03-01,2025-03-02,36,233.27,Hotel Puerta del Cielo,2.0,Madrid
11290,ca6e48aa-08af-48ae-ad2a-331b50da89c8,d7abfe3e-1e25-4c47-b90e-d79011b72855,Jordana,Iglesia,jordana.iglesia@example.com,False,2025-02-06,2025-03-01,2025-03-02,3,396.61,Hotel Las Estrellas,5.0,Madrid


In [117]:
final_estancia = pd.to_datetime(df_raw["final_estancia"][0])
inicio_estancia = pd.to_datetime(df_raw["inicio_estancia"][0])
inicio_estancia

Timestamp('2025-03-01 00:00:00')

In [132]:
info_eventos = {
    "nombre_evento": [],
    "url_evento": [],
    "codigo_postal": [],
    "direccion": [],
    "horario": [],
    "organizacion": []
}

for evento in data["@graph"]:
    start_date = pd.to_datetime(evento.get('dtstart'))
    end_date = pd.to_datetime(evento.get('dtend'))

    if start_date <= final_estancia and end_date >= inicio_estancia:
        info_eventos["nombre_evento"].append(evento.get('title'))
        info_eventos["url_evento"].append(evento.get('link'))
        address = evento.get("address", {})
        area = address.get("area", {}) 
        info_eventos["codigo_postal"].append(area.get("postal-code", np.nan))
        info_eventos["direccion"].append(area.get("street-address", np.nan))
        info_eventos["horario"].append("Del " + evento.get('dtstart').split(".")[0][:16] + " al " + evento.get('dtend').split(".")[0][:16])
        organizacion = evento.get('organization', {})
        info_eventos["organizacion"].append(organizacion.get('organization-name', np.nan))

info_eventos

{'nombre_evento': ['25º aniversario de la revista La Fragua',
  'A toda vela',
  'Acompañamiento digital a personas mayores',
  'Acompañar en la pérdida',
  'Activas en compañía: gimnasia de mantenimiento',
  'Actividades del Centro Cultural Casa del Reloj',
  'Actividades mensuales de los centros de mayores del distrito Hortaleza',
  'Amantes de las series',
  'Andrea Canepa «Wasi qaymi (Lugar y tacto)»',
  'Apoyo a la autonomía',
  'Artificios teatrales. La escena musical de los felices veinte',
  'ArTrics - Laboratorio de Creación 2024/25',
  'Asalto a la lectura. Tres cerditos y más',
  'Asombro',
  'Baile de Carnaval. Concierto de Origen',
  'Baile de Carnaval. Concierto de Versioneta',
  'Baile de Mayores en Centro Cultural Almirante Churruca',
  'Baile de Mayores en Centro Sociocultural El Greco',
  'Beethoven. Del corazón al corazón',
  'Biblioajedrez',
  'Caminamos juntas',
  'Camino hacia la autonomía financiera',
  'Campamento de carnaval y fiestas del mundo',
  'Campamentos

In [127]:
df_eventos = pd.DataFrame(info_eventos)
df_eventos

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,Del 2025-02-24 00:00 al 2025-03-02 23:59,Biblioteca Pública Municipal Iván de Vargas (C...
1,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,Del 2024-10-01 00:00 al 2025-06-01 23:59,Matadero Madrid
2,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,NaN,NaN,Del 2024-10-14 00:00 al 2025-06-30 23:59,NaN
3,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,Del 2025-01-14 17:00 al 2025-06-17 23:59,Biblioteca Pública Municipal José Saramago (Fu...
4,Activas en compañía: gimnasia de mantenimiento,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,PLAZA EURIPIDES 1,Del 2025-01-13 12:00 al 2025-03-31 23:59,Espacio de Igualdad Lucía Sánchez Saornil. San...
...,...,...,...,...,...,...
233,"Y tú, ¿cómo viajas?",http://www.madrid.es/sites/v/index.jsp?vgnextc...,28037,CALLE HERMANOS GARCIA NOBLEJAS 14,Del 2024-12-04 10:00 al 2025-05-28 23:59,Biblioteca Pública Municipal Ciudad Lineal (Ci...
234,Yoga para el buen trato II,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PASEO CHOPERA 10,Del 2025-02-18 18:00 al 2025-03-11 23:59,Espacio de Igualdad Juana Doña. Arganzuela
235,'Clases de árabe para niños',http://www.madrid.es/sites/v/index.jsp?vgnextc...,28041,AVENIDA SAN FERMIN 10,Del 2024-12-13 18:00 al 2025-05-30 23:59,Biblioteca Pública Municipal San Fermín (Usera)
236,'Refugiados' por Myriam Laguía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28019,CALLE ISAAC ALBENIZ 1,Del 2025-02-14 19:00 al 2025-03-13 23:59,Biblioteca Pública Municipal Ana María Matute ...


In [128]:
df_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nombre_evento  238 non-null    object
 1   url_evento     238 non-null    object
 2   codigo_postal  210 non-null    object
 3   direccion      210 non-null    object
 4   horario        238 non-null    object
 5   organizacion   212 non-null    object
dtypes: object(6)
memory usage: 11.3+ KB


In [129]:
df_eventos["codigo_postal"] = df_eventos["codigo_postal"].astype("Int64")
df_eventos["codigo_postal"].dtypes

Int64Dtype()